<div style="background-color: #f5f5f5; padding: 20px; border-radius: 10px; border: 1px solid #e0e0e0; text-align: center;">
  
  <img src="https://i.ibb.co/mLyt3rk/SUT-Logo.png" alt="SUT Logo" width="100">
  
  <h1 style="margin:10px 0 0 0; color:#333;">Sharif University of Technology</h1>
  <h2 style="margin:0; color:#555; font-weight:normal;">Faculty of Mathematical Sciences</h2>
  
  <hr style="border: 1px solid #ccc; margin-top:15px; margin-bottom:15px;">
  
  <h3 style="margin:10px 0 5px 0; color:#0056b3;">HW1: PGMs and Autoregressive Models</h3>
  <h4 style="margin:10px 0 5px 0; color:#0056b3;">Prepared by: Shaygan Adim</h4>
  <br/>

  <div style="font-size: 1.1em; color: #333; line-height: 1.6;">
    <strong>Course:</strong> Generative Models<br>
    <strong>Instructor:</strong> Dr. Fatemeh Seyyedsalehi<br>
    <strong>Semester:</strong> Fall 1404-1405
  </div>

</div>

<div style="background-color: #f5f5f5; border: 1px solid #f5f5f5; padding: 20px; border-radius: 10px; margin-top: 25px; margin-bottom: 25px;">
  
  <h3 style="text-align: center; margin-top: 0; color: #0056b3;">Student Information</h3>
  <p style="text-align: center; color: #555; margin-bottom: 20px; margin-top: 0;">
    Please fill in your details below.
  </p>

  <table style="width: 60%; margin-left: auto; margin-right: auto; border: none; font-size: 1.1em; color: #333;">
    <tbody>
      <tr>
        <td style="text-align: right; padding: 8px; width: 35%;"><strong>Student Name:</strong></td>
        <td style="text-align: left; padding: 8px;"><em>Your Name Here</em></td> <!-- UPDATE THIS LINE -->
      </tr>
      <tr>
        <td style="text-align: right; padding: 8px;"><strong>Student ID:</strong></td>
        <td style="text-align: left; padding: 8px;"><em>Your Student ID Here</em></td>  <!-- UPDATE THIS LINE -->
      </tr>
    </tbody>
  </table>
  
</div>

# **HW1-P1: Inference Methods in Bayesian Networks**

This assignment provides a hands-on exploration of fundamental inference algorithms in Bayesian Networks. You will implement and compare an exact inference algorithm against two different approximate (sampling-based) algorithms. The primary goal is to gain a practical understanding of their computational trade-offs, and implementation complexity.

## Assignment Overview and Structure

### What You Will Do:
In this assignment you'll be asked to implement the sections below from scratch:

- **Variable Elimination**
- **Rejection Sampling**
- **Metropolis-Hastings**
- **Comparison and Analysis**

### Learning Objectives:
- Understand the difference between exact and approximate inference
- Compare computational trade-offs of different inference algorithms
- Analyze algorithm performance


---

# **Bayesian Network: Modeling a Graduate Student's Career Prospects**

This is the Bayesian Network's structure designed to model the various factors that influence a graduate student’s ultimate career prospects.

## Network Structure & Dependencies

| Node | Description | Children |
| :--- | :--- | :--- |
| **I** | Intelligence | **P** (Preparation), **G** (Grade) |
| **D** | Difficulty | **S** (Stress) |
| **P** | Preparation | **S** (Stress), **G** (Grade) |
| **S** | Stress | **G** (Grade) |
| **G** | Grade | **L** (Letter) |
| **L** | Letter | **J** (Job Offer) |
| **J** | Job Offer | (None) |

## Node States

* **I** (Intelligence): Binary - 0 or 1
* **D** (Difficulty): Binary - 0 or 1
* **P** (Preparation): Binary - 0 or 1
* **S** (Stress): Binary - 0 or 1
* **G** (Grade): Ternary - 0, 1, or 2 (low, medium, high)
* **L** (Letter): Binary - 0 (weak) or 1 (strong)
* **J** (Job Offer): Binary - 0 or 1

You'll be asked to compute $P(I=1 | J=1, L=0)$ using the mentioned methods.

## Some Notes

* Using ChatGPT and other LLMs are allowed but you should be able to explain every line of your code completely.
* You do not need GPU for this assignment.
* I highly recommend using the exact same structure and instructions that is provided for you in the notebook, but minor changes will be tolerated.
* Read the whole notebook once before coding. It will give you a broad vision about what you should do on the whole.
* Your results should have the minimum quality of the results that already exists in the notebook.
* All of the parts that you need to implement are marked with `#TODO`.

---

## Section 1: Setup


In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import random
from collections import OrderedDict
import pandas as pd
from tqdm.auto import tqdm

np.random.seed(42)
random.seed(42)

print("Libraries imported successfully.")

Libraries imported successfully.


A simple operation counter that you'll use in the next sections to add up total number of operations.

In [1]:
class OperationCounter:
    def __init__(self):
        self.count = 0

    def add(self, n):
        self.count += n

    def reset(self):
        self.count = 0

    def __str__(self):
        return f"{self.count:,}"

op_counter = OperationCounter()
print("Operation Counter is ready.")

Operation Counter is ready.


We define our Bayesian Network as discussed previously.

In [ ]:
# Variables are represented by integers for easier indexing:
# I=0, D=1, P=2, S=3, G=4, L=5, J=6
VAR_MAP = {'I': 0, 'D': 1, 'P': 2, 'S': 3, 'G': 4, 'L': 5, 'J': 6}
REV_VAR_MAP = {v: k for k, v in VAR_MAP.items()}

# Define CPDs using numpy arrays. The dimensions correspond to the parent variables' order.
CPDs = {
    'I': np.array([0.5, 0.5]),          # P(I=1) = 0.5
    'D': np.array([0.6, 0.4]),          # P(D=1) = 0.4

    # P(P | I), Dim: (I, P)
    'P': np.array([[0.7, 0.3], [0.1, 0.9]]),

    # P(S | D, P), Dim: (D, P, S)
    'S': np.array([[[0.7, 0.3], [0.9, 0.1]], # D=0
                   [[0.2, 0.8], [0.5, 0.5]]]), # D=1

    # P(G | I, P, S), Dim: (I, P, S, G)
    'G': np.array([
        [[[0.4, 0.5, 0.1], [0.8, 0.15, 0.05]], # I=0, P=0
         [[0.2, 0.6, 0.2], [0.5, 0.4, 0.1]]], # I=0, P=1
        [[[0.3, 0.5, 0.2], [0.6, 0.3, 0.1]], # I=1, P=0
         [[0.05, 0.25, 0.7], [0.3, 0.5, 0.2]]]  # I=1, P=1
    ]),

    # P(L | G), Dim: (G, L)
    'L': np.array([[0.99, 0.01], [0.8, 0.2], [0.05, 0.95]]),

    # P(J | L), Dim: (L, J)
    'J': np.array([[0.9, 0.1], [0.2, 0.8]])
}

PARENTS = {
    'I': [], 'D': [], 'P': ['I'], 'S': ['D', 'P'], 'G': ['I', 'P', 'S'],
    'L': ['G'], 'J': ['L']
}

print("Bayesian Network structure and CPDs defined.")

Bayesian Network structure and CPDs defined.


---

## Section 2: Variable Elimination (20 Points)
In this section you'll implement variabe elimination and execute it to calculate a specific conditional probability.


### Part A: Factor Operations

**Description:**
- These helper classes and functions should implement factor operations needed for Variable Elimination
- **`Factor` Class**: Represents a factor (table) in the graphical model
- **`factor_product()` Function**: Multiplies two factors together
- **`sum_out()` Function**: Marginalizes (sums out) a variable from a factor

In [ ]:
class Factor:
    def __init__(self, variables, table, op_counter):
        """
        A factor in a graphical model.
        variables: List of variable names in the factor's scope.
        table: A numpy array representing the factor's values.
        op_counter: The global operation counter.
        """
        # Using an OrderedDict to maintain a consistent order of variables.
        self.variables = OrderedDict((var, i) for i, var in enumerate(variables))
        self.table = np.array(table)
        self.op_counter = op_counter

def factor_product(f1, f2, op_counter):
    """Computes the product of two factors using a robust broadcasting method."""
    # 1. TODO: Get the union of variables and create a final variable order
    all_vars_list = None
    all_vars_map = None

    # 2. TODO: Align f1's table to the new variable order, Add new axes for variables from f2 that are not in f1
    slice1 = None
    for var in f2.variables:
        continue
    table1_expanded = None
    
    # TODO: Permute the axes to match the all_vars_list order
    f1_order = None
    f1_permutation = None
    table1_aligned = None

    # 3. TODO: Align f2's table to the new variable order
    # Add new axes for variables from f1 that are not in f2
    slice2 = None
    for var in f1.variables:
        if var not in f2.variables:
            continue
    table2_expanded = None
    
    # TODO: Permute the axes to match the all_vars_list order
    f2_order = None
    f2_permutation = None
    table2_aligned = None

    # 4.TODO: Multiply the aligned tables
    result_table = None
    
    # Count operations: element-wise multiplication
    op_counter.add(result_table.size)
    
    return Factor(all_vars_list, result_table, op_counter)

def sum_out(factor, var_to_sum, op_counter):
    """Sums out a variable from a factor."""
    if var_to_sum not in factor.variables:
        return factor

    # Identify axis to sum over
    axis_to_sum = list(factor.variables.keys()).index(var_to_sum)
    
    # TODO: Sum over the specified axis
    result_table = None
    
    # TODO: Count operations
    additions = None
    op_counter.add(additions)

    # TODO: Create new variable list by removing the summed-out variable
    new_vars = None
    
    return Factor(new_vars, result_table, op_counter)

### Part B: Variable Elimination Algorithm

**Description:**
- This implements the Variable Elimination algorithm for exact inference
- The algorithm:
  1. Initializes factors from CPDs
  2. Sets evidence by reducing factors
  3. Eliminates variables in specified order
  4. Returns normalized probability distribution

In [ ]:
def variable_elimination(cpds, parents, query_var, evidence, elim_order, op_counter):
    """
    Performs exact inference using the Variable Elimination algorithm.
    """
    # 1. TODO: Initialize factors from CPDs
    factors = []
    for var, table in cpds.items():
        continue

    # 2. TODO: Set evidence
    for E_var, E_val in evidence.items():
        new_factors = []
        for f in factors:
            if E_var in f.variables:
                continue
        factors = new_factors

    # 3. TODO: Eliminate variables
    for var in elim_order:
        continue
        
    # 4. TODO: Final product
    final_factor = factors[0]
    for i in range(1, len(factors)):
        continue
    
    # 5. Normalization
    total_sum = np.sum(final_factor.table)
    op_counter.add(final_factor.table.size - 1)
    
    normalized_table = final_factor.table / total_sum
    op_counter.add(final_factor.table.size) 
    
    return normalized_table

### Part C: Execute Variable Elimination


In [ ]:
# --- Execute Variable Elimination ---
print("--- Running Variable Elimination ---")
ve_results = {}

op_counter.reset()
start_time = time.time()

query_variable = 'I'
evidence = {'J': 1, 'L': 0}
elimination_order = ['G', 'S', 'P', 'D']

final_dist = variable_elimination(CPDs, PARENTS, query_variable, evidence, elimination_order, op_counter)
ve_prob_I1 = final_dist[1]

end_time = time.time()

ve_results['time'] = end_time - start_time
ve_results['ops'] = op_counter.count
ve_results['prob'] = ve_prob_I1

print(f"Query: P(I=1 | J=1, L=0)")
print(f"Result: {ve_results['prob']:.6f}")
print(f"Total Arithmetic Operations: {op_counter}")
print(f"Execution Time: {ve_results['time']:.6f} seconds")

--- Running Variable Elimination ---
Query: P(I=1 | J=1, L=0)
Result: 0.345956
Total Arithmetic Operations: 89
Execution Time: 0.000000 seconds


---

## Section 3: Rejection Sampling (20 Points)

### Part A: Rejection Sampling Implementation

**Description:**
- This implements Rejection Sampling, an approximate inference method
- **How it works:**
  1. Generate complete samples from the Bayesian Network
  2. Reject samples that don't match the evidence
  3. Estimate probability from accepted samples only


In [ ]:
def generate_one_sample(cpds, parents):
    """Generates a single complete sample from the BN in topological order."""
    sample = {}
    # TODO: Topological order of variables
    order = None
    
    # TODO: generate a sample
    for var in order:
        continue
    return sample

def rejection_sampling(cpds, parents, query_var, evidence, required_samples, op_counter):
    """
    Performs inference using rejection sampling.
    """
    accepted_samples = 0
    total_samples_generated = 0
    query_true_count = 0

    pbar = tqdm(total=required_samples, desc="Rejection Sampling (accepted)", unit="acc", leave=False)
    try:
        while accepted_samples < required_samples:
            total_samples_generated += 1
            sample = generate_one_sample(cpds, parents)

            # TODO: Check if sample matches evidence
            matches = True
            for var, val in evidence.items():
                continue

            if matches:
                accepted_samples += 1
                pbar.update(1)
                if sample[query_var] == 1:
                    query_true_count += 1

                if accepted_samples % 500 == 0:
                    acc_rate = accepted_samples / max(1, total_samples_generated)
                    pbar.set_postfix(gen=total_samples_generated, acc_rate=f"{acc_rate:.4f}")
    finally:
        pbar.close()

    if accepted_samples == 0:
        return 0.0, total_samples_generated

    # Estimate probability
    op_counter.add(1)
    probability = query_true_count / accepted_samples

    return probability, total_samples_generated

### Part B: Execute Rejection Sampling

- **Parameters:**
  - `required_accepted_samples`: Require number of accepted samples

In [ ]:
print("--- Running Rejection Sampling ---")
rs_results = {}

op_counter.reset()
start_time = time.time()

required_accepted_samples = 500

rs_prob, total_generated = rejection_sampling(CPDs, PARENTS, 'I', {'J': 1, 'L': 0}, required_accepted_samples, op_counter)

end_time = time.time()

rs_results['time'] = end_time - start_time
rs_results['ops'] = op_counter.count
rs_results['prob'] = rs_prob
rs_results['total_generated'] = total_generated
rs_results['acceptance_rate'] = required_accepted_samples / total_generated if total_generated > 0 else 0

print(f"Query: P(I=1 | J=1, L=0)")
print(f"Result (estimate): {rs_results['prob']:.6f}")
print(f"Total Arithmetic Operations: {op_counter}")
print(f"Execution Time: {rs_results['time']:.6f} seconds")
print(f"\nGenerated {rs_results['total_generated']:,} total samples to get {required_accepted_samples:,} accepted samples.")
print(f"Acceptance Rate: {rs_results['acceptance_rate']:.4%}")

--- Running Rejection Sampling ---


Rejection Sampling (accepted):   0%|          | 0/500 [00:00<?, ?acc/s]

Query: P(I=1 | J=1, L=0)
Result (estimate): 0.360000
Total Arithmetic Operations: 11,123
Execution Time: 0.655196 seconds

Generated 8,164 total samples to get 500 accepted samples.
Acceptance Rate: 6.1244%


---

## Section 4: Metropolis-Hastings MCMC (20 Points)


### Part A: Metropolis-Hastings Implementation

**Description:**
- This implements Metropolis-Hastings, a Markov Chain Monte Carlo (MCMC) method
- **How it works:**
  1. Start with a random state consistent with evidence
  2. Propose changes to hidden variables (Uses symmetric proposal distribution. In other words, randomly choose different value)
  3. Accept/reject proposals based on probability ratios
  4. Collect samples after burn-in period

In [ ]:
def get_prob_of_state(state, cpds, parents, op_counter):
    """Calculates the joint probability P(I,D,P,S,G,L,J) of a given state."""
    log_prob = 0.0
    # TODO: Topological order
    order = None
    
    # TODO: Calculate the probability
    for var in order:
        continue

    op_counter.add(len(order) - 1)
    
    return np.exp(log_prob)

def metropolis_hastings(cpds, parents, query_var, evidence, iterations, burn_in, op_counter):
    """
    Performs inference using Metropolis-Hastings MCMC.
    """
    hidden_vars = [v for v in REV_VAR_MAP.values() if v not in evidence]
    
    # 1. TODO: Initialize state
    current_state = None
    current_state.update(evidence)
    
    samples = []
    running_avg_trace = []
    
    # 2. TODO: Run chain
    for i in tqdm(range(iterations), desc="Metropolis-Hastings", unit="iter", leave=False):
        # TODO: Propose a new state by flipping one hidden variable
        var_to_flip = None
        
        # TODO: For simplicity and clear op counting, re-calculate the full joint probability.
        prob_current = get_prob_of_state(current_state, cpds, parents, op_counter)
        
        proposed_state = current_state.copy()
        current_val = proposed_state[var_to_flip]
        # Determine domain size from the CPD's last dimension
        domain_size = cpds[var_to_flip].shape[-1]
        if domain_size > 2:
            # TODO: For multi-valued variables, randomly choose a different value (symmetric proposal)
            possible_values = None
            proposed_state[var_to_flip] = None
        else:
            # TODO: For binary variables, flip the value (symmetric proposal)
            proposed_state[var_to_flip] = None
        
        prob_proposed = get_prob_of_state(proposed_state, cpds, parents, op_counter)
        
        acceptance_ratio = prob_proposed / prob_current if prob_current > 0 else 1.0
        op_counter.add(1)
        
        # Accept or reject
        op_counter.add(1)
        if random.random() < acceptance_ratio:
            current_state = proposed_state
        
        # Collect sample after burn-in
        if i >= burn_in:
            samples.append(current_state[query_var])
            if len(samples) > 0:
                running_avg_trace.append(np.mean(samples))

    op_counter.add(1)
    op_counter.add(len(samples)-1)
    final_prob = np.mean(samples) if samples else 0.0
    
    return final_prob, running_avg_trace

### Part B: Execute Metropolis-Hastings

- **Parameters:**
  - `iterations`: Total number of MCMC steps
  - `burn_in`: Initial samples to discard


In [ ]:
print("--- Running Metropolis-Hastings ---")
mh_results = {}

op_counter.reset()
start_time = time.time()

iterations = 500
burn_in = 150

mh_prob, trace = metropolis_hastings(CPDs, PARENTS, 'I', {'J': 1, 'L': 0}, iterations, burn_in, op_counter)

end_time = time.time()

mh_results['time'] = end_time - start_time
mh_results['ops'] = op_counter.count
mh_results['prob'] = mh_prob
mh_results['trace'] = trace

print(f"Query: P(I=1 | J=1, L=0)")
print(f"Result (estimate): {mh_results['prob']:.6f}")
print(f"Total Arithmetic Operations: {op_counter}")
print(f"Execution Time: {mh_results['time']:.6f} seconds")

--- Running Metropolis-Hastings ---


Metropolis-Hastings:   0%|          | 0/500 [00:00<?, ?iter/s]

Query: P(I=1 | J=1, L=0)
Result (estimate): 0.351429
Total Arithmetic Operations: 14,350
Execution Time: 0.043741 seconds


---

## Section 5: Comparison and Analysis (40 Points)

**Description:**
1. Analyze the results and trade-offs
2. Answer the following questions in your analysis

### Questions to Answer:

1. **Accuracy Comparison:**
   - Compare the probability estimates from all three methods
   - Which method gives the exact answer? Which are approximate?
   - How close are the approximate methods to the exact result?

2. **Computational Cost:**
   - Compare the number of arithmetic operations for each method
   - Compare the execution times
   - Which method is most efficient? Which is least efficient?

3. **Scalability:**
   - How would each method perform with more variables?
   - How would each method handle more unlikely evidence?
   - What are the advantages and disadvantages of each approach?

4. **Stochastic Methods' Precision:**
   - Play with the parameters of Rejection Sampling and MCMC method to make their result closer to the exact inference.
   - Report the threshold parameters, execution time, and computational complexity when result differs from the exact inference by less than $1e-3$.
   - Plot the execution time and computational complexity versus the difference between the exact inference result and stochastic methods' results.